In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from test_scripts.load_images import get_video_seq
from utils.utility_functions import listdir_nohidden_sorted as lsdir
from vision.vision_utils.video_seq_generator import VideoSeqGenerator
import pandas as pd
import numpy as np
from random import randrange

In [15]:
MED_FALL = '/home/jovyan/work/MED_Fall'
GROUND_TRUTH_PATH = f'{MED_FALL}/vision/vision_dataset/ground_truth_new'
FRAMES_PATH = f'{MED_FALL}/vision/vision_dataset/extracted_frames'


In [16]:
dfs = []
for file in lsdir(GROUND_TRUTH_PATH):
    df = pd.read_csv(file)
    dfs.append(df)
dataset = pd.concat(dfs, ignore_index=True, axis=0)
dataset = dataset.iloc[:, 2:]

In [17]:
dataset

,micro_labels,macro_labels,ar_labels,frame_name
0,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0000
1,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0001
2,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0002
3,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0003
4,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0004
...,...,...,...,...
1182295,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4615
1182296,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4616
1182297,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4617
1182298,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4618


In [32]:
gen = VideoSeqGenerator(frames_path=FRAMES_PATH,
                        df = dataset,
                        seq_len = 20,
                        batch_size = 32,
                        input_shape = (224,224,3),
                       )

In [ ]:
tensor = get_video_seq(dataset["frame_name"][:20])
input_shape = tensor.shape

In [ ]:
X = np.ones((100,) + tensor.shape)
X.shape

In [ ]:
y = np.random.randint(3, size=100)
y.shape

In [ ]:
gen.input_shape

In [23]:
model = Sequential()
model.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(gen.batch_size[0], 224, 224, 3), bias_initializer=Constant(0.01)))
model.add(layers.Conv3D(32,(3,3,3),activation='relu',bias_initializer=Constant(0.01)))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Conv3D(64,(3,3,3),activation='relu'))
model.add(layers.Conv3D(64,(2,2,2),activation='relu'))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())
model.add(layers.Dense(256,'relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(128,'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3,'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 222, 222, 32)  2624      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 28, 220, 220, 32)  27680     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 14, 110, 110, 32)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 108, 108, 64)  55360     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 11, 107, 107, 64)  32832     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 53, 53, 64)     0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 53, 53, 64)     0

2022-08-21 10:19:09.754813: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-21 10:19:10.172701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21353 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [33]:
model.fit(gen, epochs=20)

<class 'tuple'> <class 'tuple'>


TypeError: can't multiply sequence by non-int of type 'tuple'